# PC Specific ScanImage --> NWB (SCRIPT)

## create NWB file

### set directories

In [1]:
import os

# dataPath = "/home/pac/Documents/Python/nwb/scanimage/rawData"
# outputNWBpath = f"/home/pac/Documents/Python/nwb/scanimage/nwbOutput/{experimentID}.nwb"

experimentID = 'AA0314'

dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304"
outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}.nwb")


experimentDir = os.path.join(dataPath,experimentID)
experiment_mat = f"{experimentID}_anmlROI_stimTable.mat"
moCorrMat = f"./NoRMCorred/{experimentID}_NoRMCorreParams.mat"
roiMat = f"{experimentID}_moCorrROI_all.mat"
fluorescenceMat = f"{experimentID}_tifFileList.mat"


pupilMat = f"{experimentID}_pulsePupilUVlegend2P_s.mat"
if (os.path.exists(os.path.join(experimentDir,pupilMat)) or 
    os.path.exists(os.path.join(experimentDir,pupilMat.replace('_s.mat','.mat')))):
    print(f"pupil data exists for {experimentID}")
    addPupillometry = True



pupil data exists for AA0314


### extract .tif image data and metadata

In [2]:
# get tif file list
import lib.mat2py

# get tif creation date, end write time, and frame counts
tifFileList = lib.mat2py.getMatCellArrayOfStr(os.path.join(experimentDir,experiment_mat),varPath = ['tifFileList','stim','name'])
fileTimeWrite = lib.mat2py.getMatCellArrayOfStr(os.path.join(experimentDir,experiment_mat),varPath = ['tifFileList','stim','date'])
tifFrameCounts = lib.mat2py.getMatCellArrayOfNum(os.path.join(experimentDir,experiment_mat),varPath = ['tifFileList','stim','nFrames'])
print(list(zip(tifFileList,fileTimeWrite,tifFrameCounts)))

[('AA0314AAAA_00010_00001.tif', '25-Apr-2021 15:33:20', 170.0), ('AA0314AAAA_00011_00001.tif', '25-Apr-2021 15:35:00', 170.0), ('AA0314AAAA_00012_00001.tif', '25-Apr-2021 15:37:22', 170.0), ('AA0314AAAA_00013_00001.tif', '25-Apr-2021 15:38:32', 170.0), ('AA0314AAAA_00014_00001.tif', '25-Apr-2021 15:39:30', 170.0), ('AA0314AAAA_00015_00001.tif', '25-Apr-2021 15:40:44', 170.0), ('AA0314AAAA_00016_00001.tif', '25-Apr-2021 15:42:11', 170.0), ('AA0314AAAA_00017_00001.tif', '25-Apr-2021 15:43:07', 170.0), ('AA0314AAAA_00018_00001.tif', '25-Apr-2021 15:44:31', 170.0), ('AA0314AAAA_00019_00001.tif', '25-Apr-2021 15:45:46', 170.0), ('AA0314AAAA_00020_00001.tif', '25-Apr-2021 15:46:46', 170.0), ('AA0314AAAA_00021_00001.tif', '25-Apr-2021 15:48:34', 170.0), ('AA0314AAAA_00022_00001.tif', '25-Apr-2021 15:49:43', 170.0), ('AA0314AAAA_00023_00001.tif', '25-Apr-2021 15:50:44', 170.0), ('AA0314AAAA_00024_00001.tif', '25-Apr-2021 15:51:42', 170.0), ('AA0314AAAA_00025_00001.tif', '25-Apr-2021 15:52:57',

In [3]:
# get tif data
import lib.tifExtract

imgData,fileTimeInstantiate,nFrames,fr = lib.tifExtract.getTifData(tifFileList, experimentDir)
sum(nFrames)

(12820, 256, 256)


12820

In [4]:
# Convert tif file instantiation date strings to timestamps of each frame in seconds
timestamps,date_times = lib.tifExtract.filetime2secTimestamp(fileTimeInstantiate, nFrames, fr)
len(timestamps)

12820

### NWB file prep

#### instantiate NWB file

In [6]:
from uuid import uuid4
from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.image import ImageSeries
from pynwb.ophys import (
    CorrectedImageStack,
    Fluorescence,
    ImageSegmentation,
    MotionCorrection,
    OpticalChannel,
    RoiResponseSeries,
    TwoPhotonSeries,
)

In [7]:
# refer to notebook for session description
# standardize: experiment_animal_mouseLine --> pupilPTinContrast_AA0304_C57BL6J
# see dataMapPCetal2024.txt: /media/DATA/backups/sutter2P_backup/D_drive

nwbfile = NWBFile(
    session_description=f"pupilPTinContrast_{experimentID}_C57BL6J",
    identifier=str(uuid4()),
    session_start_time=date_times[0],
    experimenter=[
        "Cody, Patrick",
    ],
    lab="Tzounopoulos Lab",
    institution="University of Pittsburgh",
    experiment_description="Pure tone in low and high DRC contrast with pupillometry",
    keywords=["2P", "CGC", "pupillometry", "DRC"],
    related_publications="10.1523/JNEUROSCI.0939-23.2024",
)


/home/pac/Documents/Python/nwb/venvNWB/lib/python3.12/site-packages/pynwb/file.py:472: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)


#### set imaging plane

In [8]:
device = nwbfile.create_device(
    name="Microscope",
    description="Sutter moveable objective microscope (MOM) with mode-locked laser light (MaiTai HP) at 100-200 mW intensity through 40x0.8NA objective (Olympus) with X-Y galvanometric scanning",
    manufacturer="Sutter",
)
optical_channel = OpticalChannel(
    name="OpticalChannel",
    description="green channel for GCaMP",
    emission_lambda=525.0,
)
# filter FF03 525/50 (Semrock) --> 525 +/- 25 (center wavelength 525): https://www.idex-hs.com/store/product-detail/ff03_525_50_25/fl-004656

In [9]:
imaging_plane = nwbfile.create_imaging_plane(
    name="ImagingPlane",
    optical_channel=optical_channel,
    imaging_rate=5.0,
    description="Auditory Cortex",
    device=device,
    excitation_lambda=940.0,
    indicator="GFP",
    location="ACtx",
    grid_spacing=[145.0, 145.0],
    grid_spacing_unit="micrometers",
    origin_coords=[-2.0, 4.25, 2.0],
    origin_coords_unit="meters",
)
# “Origin coordinates are relative to bregma. First dimension corresponds to anterior-posterior axis (larger index = more anterior). 
# Second dimension corresponds to medial-lateral axis (larger index = more rightward). 
# Third dimension corresponds to dorsal-ventral axis (larger index = more ventral).

#### set two-photon series data

In [10]:
# add to nwb 2p series
two_p_series = TwoPhotonSeries(
    name="TwoPhotonSeries",
    description="Raw 2p data",
    data=imgData,
    imaging_plane=imaging_plane,
    unit="normalized amplitude",
    control_description=list(zip(fileTimeInstantiate,tifFileList,nFrames,fr,fileTimeWrite)),
    comments="control_description form: (fileTimeInstantiate,file,nFrames,frameRate,fileTimeWrite)",
    timestamps=timestamps
)

nwbfile.add_acquisition(two_p_series)

In [11]:
# test file write
# def writeNWB(outputPath: str, nwbfile, overWrite: bool = True):
#     if os.path.exists(outputPath):
#         if overWrite==True:
#             os.remove(outputPath)
#         else:
#             raise('file exists')

#     with NWBHDF5IO(outputPath, "w") as io:
#         io.write(nwbfile)
from lib.nwbScanImage import writeNWB
writeNWB(outputNWBpath,nwbfile)
# view with neurosift: neurosift {outputNWBpath}


#### add motion correction data
- shifts in animalDir/NoRMCorred/[animal]_NoRMCorreParams.mat --> all
    - concatenated together of length (nTifFiles * sum(tifFileList.stim.nFrames))
    - dim of each frame is (8,8,1,2) --> can be squeezed to (8,8,2)

In [11]:
imgData_corr = lib.tifExtract.getTifData(
   [f.replace('.tif','_NoRMCorre.tif') for f in tifFileList], 
   os.path.join(experimentDir,'NoRMCorred'),
   getMetadata=False)

shifts,moCorrParams = lib.mat2py.getMoCorrShiftParams(os.path.join(experimentDir,moCorrMat))

(12820, 256, 256)


In [12]:
corrected = ImageSeries(
    name="corrected",  # this must be named "corrected"
    description="A motion corrected image stack",
    data=imgData_corr,
    unit="na",
    format="raw",
    control_description=list(zip(fileTimeInstantiate,tifFileList,nFrames,fr,fileTimeWrite)),
    comments="control_description form: (fileTimeInstantiate,file,nFrames,frameRate,fileTimeWrite)",
    timestamps=timestamps
)

xy_translation = TimeSeries(
    name="xy_translation",
    description="x,y translation in pixels",
    data=shifts,
    unit="pixels",
    timestamps=timestamps,
    control_description = moCorrParams,
    comments= 'control_description: NoRMCorreParams',
)

corrected_image_stack = CorrectedImageStack(
    corrected=corrected,
    original=two_p_series,
    xy_translation=xy_translation,
)

motion_correction = MotionCorrection(corrected_image_stacks=[corrected_image_stack])

In [ ]:
ophys_module = nwbfile.create_processing_module(
    name="ophys", description="optical physiology processed data"
)

ophys_module.add(motion_correction)

In [15]:
# test file write
writeNWB(outputNWBpath,nwbfile)
# view with neurosift: neurosift {outputNWBpath}


#### add ROI via planeSegmentation
- in experiment dir, ROI drawn on motion corrected data saved in [animal id]_moCorrROI_all.mat

In [16]:
roiMasks = lib.mat2py.getROImasks(os.path.join(experimentDir,roiMat))

In [ ]:
img_seg = ImageSegmentation()

ps = img_seg.create_plane_segmentation(
    name="PlaneSegmentation",
    description="output from segmenting the imaging plane",
    imaging_plane=imaging_plane,
    reference_images=two_p_series,  # optional
)

ophys_module.add(img_seg)

In [ ]:
from matplotlib import pyplot as plt
for roiImageMask in roiMasks:
    # add image mask to plane segmentation
    ps.add_roi(image_mask=roiImageMask)

plt.imshow(roiMasks[0])

#### add fluorescence traces for ROIs

In [ ]:
# load ROI fluo data from experiment
arrL = lib.mat2py.getROIfluo(os.path.join(experimentDir,fluorescenceMat))
arrL.shape

In [ ]:
rt_region = ps.create_roi_table_region(
    region=list(range(arrL.shape[1])), description="all ROI"
)

roi_resp_series = RoiResponseSeries(
    name="RoiResponseSeries",
    description="Fluorescence responses for motion corrected ROIs",
    data=arrL,
    rois=rt_region,
    unit="lumens",
    rate=5.0,
)

fl = Fluorescence(roi_response_series=roi_resp_series)
ophys_module.add(fl)

In [21]:
# test file write
writeNWB(outputNWBpath,nwbfile)
# view with neurosift: neurosift {outputNWBpath}

#### add sound stimulus data via DynamicTable

In [22]:
pulseNames = lib.mat2py.getMatCellArrayOfStr(os.path.join(experimentDir,experiment_mat),['pulseLegend2P','pulseName'])
pulseSets = lib.mat2py.getMatCellArrayOfStr(os.path.join(experimentDir,experiment_mat),['pulseLegend2P','pulseSet'])

stimData = {
    'fileTimeInstantiate': ('time .tif file was instantiated/created',fileTimeInstantiate),
    'file': ('name of .tif file',tifFileList),
    'nFrames': ('number of frames in .tif file',nFrames),
    'frameRate': ('frame rate of .tif file',fr),
    'pulseNames': ('sound stimulation pulse name',pulseNames),
    'pulseSets': ('sound stimulation pulse set',pulseSets)
}

In [23]:
from hdmf.common import VectorData, DynamicTable

cols = []
for col,v in stimData.items():
    cols.append(
            VectorData(
            name=col,
            description=v[0],
            data=v[1],
        )
    )
stim_table = DynamicTable(
    name='stim param table',
    description='Maps sound stim parameters to .tif files',
    columns=cols,
)

nwbfile.add_stimulus(stim_table)

In [24]:
# test file write
writeNWB(outputNWBpath,nwbfile)
# view with neurosift: neurosift {outputNWBpath}

if addPupillometry==False:
    print('stop here, no pupillometry')

#### add pupillometry data
- behavior/pupil tracking with timeSeries for pupilDiameter
- processing/imageSeries for pupil video

In [25]:
pupilData = lib.mat2py.getPupilData(pupilMat,getImgData=True,experimentDir=experimentDir)
# if not adding pupil video to NWB
# pupilData = lib.mat2py.getPupilData(pupilMat,getImgData=False,experimentDir=experimentDir)

##### get pupil timestamps

In [26]:
pupilTimestamps,pupilDate_times = lib.tifExtract.filetime2secTimestamp(fileTimeInstantiate,pupilData['nFrames'],pupilData['frameRate'])

##### add pupil radius

In [27]:
import numpy as np
from pynwb.behavior import PupilTracking

behavior_module = nwbfile.create_processing_module(
    name="behavior", description="Processed behavioral data"
)

pupil_diameter = TimeSeries(
    name="pupil_radius",
    description="Pupil radius extracted from the video of the right eye.",
    data=np.concatenate(pupilData['pupilRadius']),
    timestamps=pupilTimestamps,
    unit="na",
)

pupil_tracking = PupilTracking(time_series=pupil_diameter, name="PupilTracking")

behavior_module.add(pupil_tracking)

In [28]:
# test file write
writeNWB(outputNWBpath,nwbfile)
# view with neurosift: neurosift {outputNWBpath}

##### add pupil video
- without video: ~1.4 GB for AA0304
- with pupil video: ~4.6 GB for AA0304

In [29]:
pupil_video = ImageSeries(
    name="pupil_video",
    description="Pupil video of the right eye.",
    data=pupilData['pupilImgData'],
    timestamps=pupilTimestamps,
    unit="arbitrary",
)

pupil_video_module = nwbfile.create_processing_module(
    name="pupillometry video", description="pupillometry video data"
)

pupil_video_module.add(pupil_video)

In [30]:
# test file write
writeNWB(outputNWBpath,nwbfile)
# view with neurosift: neurosift {outputNWBpath}

# ScanImage --> NWB function

In [6]:
import lib.nwbScanImage

import importlib
importlib.reload(lib.nwbScanImage)

experimentID = 'AA0314'

dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304"
outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}.nwb")

session_description = f"pupilPTinContrast_{experimentID}_C57BL6J"
experiment_description = "Pure tone in low and high DRC contrast with pupillometry"
keywords = ["2P", "CGC", "pupillometry", "DRC"]

lib.nwbScanImage.genNWBfromScanImage_pc(
    experimentID=experimentID, 
    dataPath=dataPath, 
    NWBoutputPath=outputNWBpath,
    session_description=session_description,
    experiment_description=experiment_description,
    keywords=keywords,
    **lib.nwbScanImage.PARAMS_nwbFilePC,
    **lib.nwbScanImage.PARAMS_imagingPC
)

[('AA0314AAAA_00010_00001.tif', '25-Apr-2021 15:33:20', 170.0), ('AA0314AAAA_00011_00001.tif', '25-Apr-2021 15:35:00', 170.0), ('AA0314AAAA_00012_00001.tif', '25-Apr-2021 15:37:22', 170.0), ('AA0314AAAA_00013_00001.tif', '25-Apr-2021 15:38:32', 170.0), ('AA0314AAAA_00014_00001.tif', '25-Apr-2021 15:39:30', 170.0), ('AA0314AAAA_00015_00001.tif', '25-Apr-2021 15:40:44', 170.0), ('AA0314AAAA_00016_00001.tif', '25-Apr-2021 15:42:11', 170.0), ('AA0314AAAA_00017_00001.tif', '25-Apr-2021 15:43:07', 170.0), ('AA0314AAAA_00018_00001.tif', '25-Apr-2021 15:44:31', 170.0), ('AA0314AAAA_00019_00001.tif', '25-Apr-2021 15:45:46', 170.0), ('AA0314AAAA_00020_00001.tif', '25-Apr-2021 15:46:46', 170.0), ('AA0314AAAA_00021_00001.tif', '25-Apr-2021 15:48:34', 170.0), ('AA0314AAAA_00022_00001.tif', '25-Apr-2021 15:49:43', 170.0), ('AA0314AAAA_00023_00001.tif', '25-Apr-2021 15:50:44', 170.0), ('AA0314AAAA_00024_00001.tif', '25-Apr-2021 15:51:42', 170.0), ('AA0314AAAA_00025_00001.tif', '25-Apr-2021 15:52:57',

/home/pac/Documents/Python/nwb/venvNWB/lib/python3.12/site-packages/pynwb/file.py:472: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)


(12820, 256, 256)
added motion correction data


FileNotFoundError: [Errno 2] No such file or directory: '/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304/AA0314/AA0314_moCorrROI_all.mat'

In [1]:
import os
import lib.nwbScanImage

import importlib
importlib.reload(lib.nwbScanImage)

experimentID = 'AA0308'
# /media/DATA/backups/sutter2P_backup/D_drive/ZnT3_pupil/BIN2/AA0323
#  /media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304/AA0308
dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304"
outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}.nwb")

session_description = f"pupilPTinContrast_{experimentID}_C57BL6J"
experiment_description = "Pure tone in low and high DRC contrast with pupillometry"
keywords = ["2P", "CGC", "pupillometry", "DRC"]

lib.nwbScanImage.genNWBfromScanImage_pc(
    experimentID=experimentID, 
    dataPath=dataPath, 
    NWBoutputPath=outputNWBpath,
    session_description=session_description,
    experiment_description=experiment_description,
    keywords=keywords,
    **lib.nwbScanImage.PARAMS_nwbFilePC,
    **lib.nwbScanImage.PARAMS_imagingPC
)

[('AA0308AAAA_00020_00001.tif', '14-Apr-2021 17:32:03', 170.0), ('AA0308AAAA_00021_00001.tif', '14-Apr-2021 17:34:24', 170.0), ('AA0308AAAA_00022_00001.tif', '14-Apr-2021 17:37:16', 170.0), ('AA0308AAAA_00023_00001.tif', '14-Apr-2021 17:39:03', 170.0), ('AA0308AAAA_00025_00001.tif', '14-Apr-2021 17:44:17', 170.0), ('AA0308AAAA_00026_00001.tif', '14-Apr-2021 17:45:37', 170.0), ('AA0308AAAA_00027_00001.tif', '14-Apr-2021 17:46:54', 170.0), ('AA0308AAAA_00028_00001.tif', '14-Apr-2021 17:50:36', 120.0), ('AA0308AAAA_00029_00001.tif', '14-Apr-2021 17:53:07', 120.0), ('AA0308AAAA_00030_00001.tif', '14-Apr-2021 17:54:23', 120.0), ('AA0308AAAA_00031_00001.tif', '14-Apr-2021 17:56:11', 120.0), ('AA0308AAAA_00032_00001.tif', '14-Apr-2021 17:57:00', 120.0), ('AA0308AAAA_00033_00001.tif', '14-Apr-2021 17:57:48', 120.0), ('AA0308AAAA_00034_00001.tif', '14-Apr-2021 17:58:39', 120.0), ('AA0308AAAA_00035_00001.tif', '14-Apr-2021 17:59:27', 120.0), ('AA0308AAAA_00036_00001.tif', '14-Apr-2021 18:02:30',

/home/pac/Documents/Python/nwb/venvNWB/lib/python3.12/site-packages/pynwb/file.py:472: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)


(7460, 256, 256)
added motion correction data
added ROI segmentation data


/home/pac/Documents/Python/nwb/venvNWB/lib/python3.12/site-packages/hdmf/container.py:536: UserWarning: The linked table for DynamicTableRegion 'rois' does not share an ancestor with the DynamicTableRegion.
  child._validate_on_set_parent()


added fluorescence trace data for ROIs
added stim table data
found pupillometry data for AA0308
added pupil radius data
added pupil video data
NWB write success to: /media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304/AA0308/AA0308.nwb


### loop over dirs

In [1]:
import os
import lib.nwbScanImage

import importlib
importlib.reload(lib.nwbScanImage)

dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304"

experiments = ['AA0310',
               'AA0312',
               'AA0314',
            #    'AA0315',
            #    'AA0318',
            #    'AA0350',
            #    'AA0353'
               ]

for experimentID in experiments:
    outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}.nwb")

    if os.path.exists(outputNWBpath):
        print(f"NWB file already created for {experimentID}")
    else:
        session_description = f"pupilPTinContrast_{experimentID}_C57BL6J"
        experiment_description = "Pure tone in low and high DRC contrast with pupillometry"
        keywords = ["2P", "CGC", "pupillometry", "DRC"]

        lib.nwbScanImage.genNWBfromScanImage_pc(
            experimentID=experimentID, 
            dataPath=dataPath, 
            NWBoutputPath=outputNWBpath,
            session_description=session_description,
            experiment_description=experiment_description,
            keywords=keywords,
            **lib.nwbScanImage.PARAMS_nwbFilePC,
            **lib.nwbScanImage.PARAMS_imagingPC
            )

NWB file already created for AA0310
NWB file already created for AA0312
[('AA0314AAAA_00010_00001.tif', '25-Apr-2021 15:33:20', 170.0), ('AA0314AAAA_00011_00001.tif', '25-Apr-2021 15:35:00', 170.0), ('AA0314AAAA_00012_00001.tif', '25-Apr-2021 15:37:22', 170.0), ('AA0314AAAA_00013_00001.tif', '25-Apr-2021 15:38:32', 170.0), ('AA0314AAAA_00014_00001.tif', '25-Apr-2021 15:39:30', 170.0), ('AA0314AAAA_00015_00001.tif', '25-Apr-2021 15:40:44', 170.0), ('AA0314AAAA_00016_00001.tif', '25-Apr-2021 15:42:11', 170.0), ('AA0314AAAA_00017_00001.tif', '25-Apr-2021 15:43:07', 170.0), ('AA0314AAAA_00018_00001.tif', '25-Apr-2021 15:44:31', 170.0), ('AA0314AAAA_00019_00001.tif', '25-Apr-2021 15:45:46', 170.0), ('AA0314AAAA_00020_00001.tif', '25-Apr-2021 15:46:46', 170.0), ('AA0314AAAA_00021_00001.tif', '25-Apr-2021 15:48:34', 170.0), ('AA0314AAAA_00022_00001.tif', '25-Apr-2021 15:49:43', 170.0), ('AA0314AAAA_00023_00001.tif', '25-Apr-2021 15:50:44', 170.0), ('AA0314AAAA_00024_00001.tif', '25-Apr-2021 1

/home/pac/Documents/Python/nwb/venvNWB/lib/python3.12/site-packages/pynwb/file.py:472: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)


(12820, 256, 256)
added motion correction data


FileNotFoundError: [Errno 2] No such file or directory: '/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304/AA0314/AA0314_moCorrROI_all.mat'